### PARTIE QUESTIONS DETAILLEES

In [11]:
import os
import json
import httpx
from dotenv import load_dotenv

# Charger la clé depuis le fichier .env
load_dotenv()
api_key = os.getenv("MISTRAL_API_KEY")

# Préparer la requête
url = "https://api.mistral.ai/v1/chat/completions"

In [12]:
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

payload = {
    "model": "mistral-tiny",
    "messages": [
        {"role": "user", "content": "Dis bonjour en français"}
    ],
    "temperature": 0.7,
    "max_tokens": 100
}

# Envoi de la requête
try:
    response = httpx.post(url, headers=headers, data=json.dumps(payload), timeout=15)
    result = response.json()
    print("✅ Réponse :")
    print(result["choices"][0]["message"]["content"])

except Exception as e:
    print("❌ Erreur lors de l'appel à Mistral :")
    print(e)


✅ Réponse :
❌ Erreur lors de l'appel à Mistral :
'choices'


In [22]:
import os
import json
import httpx
from dotenv import load_dotenv
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output

# Charger la clé API
load_dotenv()
api_key = os.getenv("MISTRAL_API_KEY")
url = "https://api.mistral.ai/v1/chat/completions"
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

# Historique pour conversation continue
historique_messages = [
    {"role": "system", "content": "Tu es un assistant de voyage expert."}
]

# UI formulaire initial
display(Markdown("### 👋 Bienvenue dans l'assistant de voyage intelligent !"))
display(Markdown("Je vais te poser quelques questions pour mieux cerner tes envies de voyage 🌍"))

# Widgets initiaux
destination = widgets.Text(placeholder="ex : plage, montagne, ville...", description="")
budget = widgets.Text(placeholder="ex : faible, 500€, moyen...", description="")
dates = widgets.Text(placeholder="ex : du 15 au 22 juillet", description="")
interets = widgets.Text(placeholder="ex : randonnée, musées, gastronomie...", description="")
hebergement = widgets.Text(placeholder="ex : hôtel, auberge, logement insolite...", description="")
compagnie = widgets.Text(placeholder="ex : seul, en couple, en famille...", description="")
contraintes = widgets.Text(placeholder="ex : animaux, enfants, mobilité réduite...", description="")

# Affichage des questions
for i, (label, widget) in enumerate([
    ("Quel type de destination préfères-tu ?", destination),
    ("Quel est ton budget approximatif ?", budget),
    ("À quelles dates veux-tu partir ?", dates),
    ("Quelles activités t’intéressent ?", interets),
    ("Quel type d’hébergement préfères-tu ?", hebergement),
    ("Avec qui voyages-tu ?", compagnie),
    ("As-tu des contraintes particulières ?", contraintes),
]):
    display(Markdown(f"**{i+1}. {label}**"))
    display(widget)

# Bouton de validation
bouton = widgets.Button(description="✨ Générer ma recommandation ✈️")

# Zone pour le chat + widgets
zone_chat = widgets.Output()
zone_input = widgets.Textarea(placeholder="Pose une autre question, ou réagis à la réponse…", layout=widgets.Layout(width='100%', height='60px'))
bouton_suivant = widgets.Button(description="Envoyer 💬")

# Fonction de réponse continue
def continuer_conversation(b):
    user_input = zone_input.value.strip()
    if not user_input:
        return

    zone_input.value = ""
    with zone_chat:
        display(Markdown(f"**👤 Toi :** {user_input}"))

    historique_messages.append({"role": "user", "content": user_input})

    payload = {
        "model": "mistral-tiny",
        "messages": historique_messages,
        "temperature": 0.8,
        "max_tokens": 700
    }

    try:
        response = httpx.post(url, headers=headers, data=json.dumps(payload), timeout=30)
        content = response.json()["choices"][0]["message"]["content"]
        historique_messages.append({"role": "assistant", "content": content})
        with zone_chat:
            display(Markdown(f"**🤖 Assistant :** {content}"))

    except Exception as e:
        with zone_chat:
            display(Markdown("❌ **Erreur lors de l'appel à Mistral.**"))
            print(e)

# Associer la suite
bouton_suivant.on_click(continuer_conversation)

# Fonction d’appel initial
def on_button_clicked(b):
    clear_output(wait=True)
    display(Markdown("### ✈️ Génération de ta recommandation..."))

    profil = f"""
    Type de destination : {destination.value}
    Budget : {budget.value}
    Dates : {dates.value}
    Centres d’intérêt : {interets.value}
    Hébergement souhaité : {hebergement.value}
    Compagnie : {compagnie.value}
    Contraintes : {contraintes.value}
    """

    prompt = f"""
Voici un profil utilisateur :
{profil}

Tu dois recommander :
- une destination
- un itinéraire (si possible)
- 2 ou 3 activités principales
- des suggestions d’hébergement
- une justification de tes choix
- des options alternatives (budget/premium) si pertinent

Réponds en français.
"""

    historique_messages.append({"role": "user", "content": prompt})

    payload = {
        "model": "mistral-tiny",
        "messages": historique_messages,
        "temperature": 0.8,
        "max_tokens": 700
    }

    try:
        response = httpx.post(url, headers=headers, data=json.dumps(payload), timeout=30)
        content = response.json()["choices"][0]["message"]["content"]
        historique_messages.append({"role": "assistant", "content": content})
        display(Markdown("### ✅ Recommandation générée :"))
        display(Markdown(f"```\n{content}\n```"))

        # Afficher la suite : chat
        display(Markdown("---"))
        display(Markdown("### 💬 Tu peux continuer la discussion :"))
        display(zone_chat)
        display(zone_input)
        display(bouton_suivant)

    except Exception as e:
        display(Markdown("❌ **Erreur lors de l'appel à Mistral :**"))
        print(e)

# Attacher au bouton
bouton.on_click(on_button_clicked)
display(bouton)

### ✈️ Génération de ta recommandation...

### ✅ Recommandation générée :

```
Destination recommandée : Costa del Sol en Espagne. Elle offre des plages magnifiques et une ambiance agréable en été. Son climat est sec et ensoleillé, idéal pour les randonnées.

Itinéraire :
1. Jour 1 : Arrivée à Málaga et installation à l'hôtel.
2. Jour 2-3 : Randonnées au Parc National de Sierra Nevada, où vous trouverez des sentiers adaptés à tous les niveaux.
3. Jour 4-5 : Visite de la ville de Marbella, avec ses plages et son centre-ville pittoresque. Une excursion à la Réserve naturelle de La Reserva del Ojen serait un excellent complément.
4. Jour 6 : Jour de repos sur la plage de Benalmadena ou un bateau-mouche vers Puerto Banús pour la croisière et la visite du port.
5. Jour 7 : Sortie vers la Réserve de la Sierra de las Nieves, un espace naturel protégé abritant des espèces rares.
6. Jour 8 : Depart et retour à Málaga pour le retour.

Activités principales :
1. Randonnées dans les parcs naturels de Sierra Nevada et Sierra de las Nieves.
2. Visite de la ville de Marbella et de sa plage.
3. Excursion en bateau-mouche vers Puerto Banús.

Suggestions d'hébergement :
- Premium : Hôtel Vincci Selección Posada del Patio, Málaga. Situé dans le centre-ville, cet hôtel offre des chambres spacieuses et confortables, une piscine et un restaurant gastronomique.
- Budget : Hôtel Bahía Real Torrequebrada, Benalmadena. Offrant une belle vue sur la mer, cet hôtel est équipé de piscine, restaurant, bar et un centre de wellness.

Justification : La Costa del Sol est une destination populaire pour les couples qui aiment les plages et l'agreed'espace. Elle offre des opportunités de randonnées dans des parcs naturels adjacents, ce qui répond à l'intérêt de ceux qui aiment la randonnée. L'hébergement choisi est adapté aux goûts du couple et offre des services comme la piscine, le restaurant et le centre de wellness.

Options alternatives (budget/premium) :
- Pour un budget plus restreint, l'hôtel Iberostar Málaga Playa est une option de qualité/prix intéressante.
- Pour un choix de luxe, le Finca Cortesín, situé à Casares entre Marbella et Sotogrande, offre un niveau d'hébergement exceptionnel.
```

---

### 💬 Tu peux continuer la discussion :

Output()

Textarea(value='', layout=Layout(height='60px', width='100%'), placeholder='Pose une autre question, ou réagis…

Button(description='Envoyer 💬', style=ButtonStyle())

### PARTIE NON GUIDEE

Test Mistral (comparé à Ollama avec le même prompt) :

In [2]:
import os
import json
import httpx
from dotenv import load_dotenv
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output

# 🔐 Charger la clé API
load_dotenv()
api_key = os.getenv("MISTRAL_API_KEY")

# 🔗 Configuration API
url = "https://api.mistral.ai/v1/chat/completions"
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

# 🧠 Historique conversationnel
conversation = [
    {"role": "system", "content": "Tu es un assistant expert en voyages. Pose des questions si nécessaire, et fais des suggestions claires et adaptées."}
]

# 🧱 Interface utilisateur
chat_box = widgets.Output()
user_input = widgets.Textarea(
    placeholder="Pose une question ou continue la conversation...",
    layout=widgets.Layout(width="100%", height="80px")
)
send_button = widgets.Button(description="Envoyer 💬", button_style="primary")

# 📤 Fonction d’envoi
def envoyer_message(b):
    prompt = user_input.value.strip()
    if not prompt:
        return

    user_input.value = ""  # Réinitialiser l'input
    conversation.append({"role": "user", "content": prompt})
    with chat_box:
        display(Markdown(f"**👤 Toi :** {prompt}"))

    payload = {
        "model": "mistral-tiny",  # Ou mistral-small si dispo
        "messages": conversation[-10:],  # limiter le contexte si besoin
        "temperature": 0.7,
        "max_tokens": 600
    }

    try:
        with chat_box:
            print("⏳ Mistral réfléchit...")

        response = httpx.post(url, headers=headers, data=json.dumps(payload), timeout=30)
        response.raise_for_status()
        result = response.json()
        answer = result["choices"][0]["message"]["content"]
        conversation.append({"role": "assistant", "content": answer})

        with chat_box:
            clear_output(wait=True)
            for msg in conversation[1:]:  # on saute le system
                role = "👤 Toi" if msg["role"] == "user" else "🤖 Mistral"
                display(Markdown(f"**{role} :** {msg['content']}"))

    except Exception as e:
        with chat_box:
            display(Markdown(f"❌ **Erreur :** {e}"))

# 🔁 Lier le bouton à la fonction
send_button.on_click(envoyer_message)

# 📋 Affichage complet
display(Markdown("## 💬 Assistant Mistral – Chat en direct"))
display(chat_box)
display(user_input)
display(send_button)

## 💬 Assistant Mistral – Chat en direct

Output()

Textarea(value='', layout=Layout(height='80px', width='100%'), placeholder='Pose une question ou continue la c…

Button(button_style='primary', description='Envoyer 💬', style=ButtonStyle())